# Ninety One Technical Assessment – Portfolio Risk Analysis  
### *Prepared by Nicholas de Clercq*

This notebook explores the characteristics, risk dynamics, and ESG profile of an equity portfolio provided for analysis.  
All analytics and visualisations are performed in **Python**, with results designed to demonstrate both technical proficiency and interpretive insight.

---

## Project Overview

The analysis is structured in four parts:

1. **Current Portfolio Characteristics**  
   - Composition by sector, country, and position size  
   - Concentration and diversification metrics  
   - Activeness versus the benchmark  

2. **Return Analysis**  
   - Returns  
   - Active returns
   - Fund size through time
   - Return attribution  
     - Top 10 share contributors and detractors  
     - Sector return attribution  
     - Region return attribution  
   - Sector exposures through time  
   - Regional exposures through time  

3. **Risk Analysis**  
   - Total risk and active risk trends  
   - Factor exposures and beta evolution  
   - Contribution to risk by sector, region, and asset  
   - Portfolio diversification and correlation insights  
   - drawdowns, sharpe ratio, rolling standard deviation, VaR, cVaR ...

4. **ESG Analysis**  
   - Weighted ESG scores and trends  
   - Breakdown of Environmental, Social, and Governance dimensions  
   - Relationship between ESG quality and risk contribution  

---

### Objective

The goal is to identify and clearly articulate the **key drivers of portfolio risk, performance, and ESG trends** —  
and to demonstrate how these factors have evolved over time.

---

*All charts are interactive and were generated using `plotly.express` with a custom Ninety One colour palette.*


In [86]:
import importlib, portfolio_utils as pu
importlib.reload(pu)
import warnings
warnings.filterwarnings('ignore')
# jupyter nbconvert --to html --no-input "Optimise v1.ipynb"
#  jupyter nbconvert --to html --no-input "C:\Users\nicholas.declercq\OneDrive - Prescient\Optimser jupyter\Optimise V2.ipynb"

from portfolio_utils import load_data, add_derived_columns, PortfolioAnalyzer, top10_pie_latest, portfolio_vs_benchmarks, compute_portfolio_benchmark_returns
print("OK:", load_data, PortfolioAnalyzer)

OK: <function load_data at 0x0000026B4FE39440> <class 'portfolio_utils.PortfolioAnalyzer'>


In [87]:
df = load_data("Risk Analyst Case study_092025.csv")  # or "data/..." if you moved it
df = add_derived_columns(df)
pa = PortfolioAnalyzer(df)
pa.timeseries()


,refdate,Portfolio MV,Total Weight,Total Active Weight,Avg Total Risk,Wgt Risk Sum,Avg Beta,Avg ESG (norm),Avg ESG
0,2022-09-29,2.955585e+07,1.0001,0.0040,41.583675,37.049888,0.851507,0.396077,3.960768
1,2022-09-30,2.961014e+07,1.0001,0.0034,41.572124,37.283538,0.853040,0.396077,3.960768
2,2022-10-03,3.006451e+07,1.0000,0.0032,41.574890,37.879639,0.851136,0.396077,3.960768
3,2022-10-04,3.079677e+07,0.9999,0.0032,41.574890,38.089748,0.849895,0.395915,3.959147
4,2022-10-05,3.136134e+07,0.9997,0.0024,41.613675,38.576288,0.852139,0.396746,3.967462
...,...,...,...,...,...,...,...,...,...
131,2023-03-24,3.932928e+07,1.0000,0.0030,40.002393,38.822495,0.832892,0.403424,4.034241
132,2023-03-27,3.902951e+07,0.9998,0.0046,40.002796,38.817521,0.833512,0.403316,4.033158
133,2023-03-28,3.926169e+07,1.0003,0.0041,39.984457,38.737198,0.833060,0.403116,4.031156
134,2023-03-29,3.985629e+07,1.0000,0.0033,39.908214,38.796482,0.828656,0.403058,4.030578


In [88]:
top10_pie_latest(df, by="Active Weight (%)").show()

In [89]:
fig, tri_df = portfolio_vs_benchmarks(df)
fig.show()

In [94]:
# weight coverage after lag should be ~1.0 each date
d = df.sort_values(["Asset Name", "refdate"]).copy()
tmp = d.groupby("refdate")["w_lag"].sum().reset_index(name="w_lag_sum")
print(tmp.describe())
print("Min dates:", tmp.nsmallest(5, "w_lag_sum"))

# max/min stock daily returns used
print(d["ret_i"].describe())


KeyError: 'Column not found: w_lag'